# EDA WITH SQLite (Spacex)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
%matplotlib inline

In [4]:
df = pd.read_csv("C:/Users/qluis/Downloads/spacexsql.csv")
df.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [120]:
df.dtypes

Date                 object
Time (UTC)           object
Booster_Version      object
Launch_Site          object
Payload              object
PAYLOAD_MASS__KG_     int64
Orbit                object
Customer             object
Mission_Outcome      object
Landing_Outcome      object
dtype: object

In [6]:
%load_ext sql

In [8]:
con = sqlite3.connect('spacex.db')
cur = con.cursor()

In [10]:
%sql sqlite:///spacex.db

In [12]:
df.to_sql("spacex", con, if_exists='replace', index=False)

101

### Task 1


##### Display the names of the unique launch sites  in the space mission


In [16]:
%sql select distinct Launch_Site from spacex

 * sqlite:///spacex.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [39]:
%%sql
select Launch_Site from spacex
where Launch_Site like 'CCA%'
limit (5)

 * sqlite:///spacex.db
Done.


Launch_Site
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40


### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [57]:
%%sql
select sum (PAYLOAD_MASS__KG_) from spacex
where Customer = 'NASA (CRS)'

 * sqlite:///spacex.db
Done.


sum (PAYLOAD_MASS__KG_)
45596


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [66]:
%%sql 
select AVG (PAYLOAD_MASS__KG_) from spacex
where Booster_Version like 'F9 v1.1%'

 * sqlite:///spacex.db
Done.


AVG (PAYLOAD_MASS__KG_)
2534.6666666666665


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 

In [70]:
%%sql 
select min (Date) as 'Fist Successful Landing' from spacex
where Mission_Outcome = 'Success'

 * sqlite:///spacex.db
Done.


Fist Successful Landing
2010-06-04


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [78]:
%%sql select distinct Booster_Version from spacex
where Landing_Outcome = 'Success (drone ship)'  and PAYLOAD_MASS__KG_ between 4000 and 6000;

 * sqlite:///spacex.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


### Task 7


##### List the total number of successful and failure mission outcomes


In [106]:
%%sql 
select Mission_Outcome, count(*) as total_number from spacex
group by Mission_Outcome;

 * sqlite:///spacex.db
Done.


Mission_Outcome,total_number
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


In [108]:
%sql select distinct Mission_Outcome from spacex

 * sqlite:///spacex.db
Done.


Mission_Outcome
Success
Failure (in flight)
Success (payload status unclear)
Success


### Task 8



##### List all the booster_versions that have carried the maximum payload mass. Use a subquery.


In [110]:
%%sql
select Booster_Version from spacex
where PAYLOAD_MASS__KG_ = (select MAX (PAYLOAD_MASS__KG_) from spacex)

 * sqlite:///spacex.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**

In [124]:
%%sql 
SELECT 
    CASE substr(Date, 6, 2)
        WHEN '01' THEN 'January'
        WHEN '02' THEN 'February'
        WHEN '03' THEN 'March'
        WHEN '04' THEN 'April'
        WHEN '05' THEN 'May'
        WHEN '06' THEN 'June'
        WHEN '07' THEN 'July'
        WHEN '08' THEN 'August'
        WHEN '09' THEN 'September'
        WHEN '10' THEN 'October'
        WHEN '11' THEN 'November'
        WHEN '12' THEN 'December'
    END AS Month,
    Landing_Outcome,
    Booster_Version,
    Launch_Site
FROM spacex
WHERE 
    substr(Date, 0, 5) = '2015'
    AND Landing_Outcome LIKE '%failure%'
    AND Landing_Outcome LIKE '%drone ship%'

 * sqlite:///spacex.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site
January,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [142]:
%%sql 
select Landing_Outcome, count(*) as count_outcomes from spacex
where Date between '2010-06-04' and '2017-03-20'
group by Landing_Outcome
order by count_outcomes desc;

 * sqlite:///spacex.db
Done.


Landing_Outcome,count_outcomes
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
